In [ ]:
import os
from src.main import ClusterService
os.chdir('..')

# ClusterService

This class is used for clustering the images and evaluation results

In [ ]:
service = ClusterService(path_to_images='data/processed/test-task/clusters', path_to_target_clusters='data/processed/test-task/clusters.csv')

Target clusters contain different cluster_id with background images. But model for generating embeddings of faces ignore background images. That's why full evaluation according to 'clusters.csv' is poor. However, after face filtering images and clustering them, quality of algorithm dramatically improved 

In [3]:
service.run()


Background and face images clustering: 
Homogeneity score: 0.3965393867896544
Completeness score: 0.6129324424069272
V-Measure score: 0.4815426202610689

Face images clustering: 
Homogeneity score: 0.9768697821391191
Completeness score: 0.8898553170854854
V-Measure score: 0.9313345281506219
